## Gradio - ML작업 UI

---

## ■ 유사 키워드 맵핑 결과 감정별 출력

In [3]:
cd "./Beer_Sentiment_analysis/Data"

c:\Users\Gyeom\OneDrive - 한국항공대학교\바탕 화면\머신러닝\프로젝트\맥주 측면 감정 분석\Beer_Sentiment_analysis\Data


In [ ]:
from collections import defaultdict
import spacy

nlp = spacy.load('en_core_web_lg')

def merge_similar_keywords(dataframe, similarity_threshold=0.7):
    # (csv파일을 읽은 데이터프레임일 때) 문자열로 표현된 리스트를 실제 리스트로 변환
    dataframe['Keywords_List'] = dataframe['Keywords'].apply(lambda x: ast.literal_eval(x))
    
    # 모든 키워드 리스트를 하나로 합침
    all_keywords = dataframe['Keywords'].sum()

    initial_counts = defaultdict(int)
    for keyword in all_keywords:
        initial_counts[keyword] += 1

    merged_keywords = defaultdict(int)
    for keyword, count in initial_counts.items():
        doc1 = nlp(keyword)

        merged_or_found_similar = False
        for merged_keyword in list(merged_keywords):
            # 키워드가 일치하는 경우 바로 빈도 수를 올림
            if keyword == merged_keyword:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                break

            doc2 = nlp(merged_keyword)
            if doc1.similarity(doc2) > similarity_threshold:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                break

        if not merged_or_found_similar:
            merged_keywords[keyword] = count

    # (csv파일을 읽은 데이터프레임일 때) 변환된 'Keywords_List' 컬럼 삭제
    dataframe.drop('Keywords_List', axis=1, inplace=True)

    # 빈도 수 차등에 따라 정렬한 결과를 반환(내림차순)
    sorted_merged_keywords = sorted(merged_keywords.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_merged_keywords)


### - 시도1_집계만 출력

In [ ]:
import pandas as pd
import gradio as gr

pre_apply_beer_Wired_iStout = pd.read_csv('pp_selected_reviews_JJ_NN.csv')
df_po = pre_apply_beer_Wired_iStout[pre_apply_beer_Wired_iStout['MultinomialNB_label'] == "Positive"]
df_po.reset_index(drop=True, inplace=True)
df_ne = pre_apply_beer_Wired_iStout[pre_apply_beer_Wired_iStout['MultinomialNB_label'] == "Negative"]
df_ne.reset_index(drop=True, inplace=True)

def show_keywords(beer_name, sentiment):
    if sentiment == 'Positive': df = df_po
    else: df = df_ne

    keywords, _ = merge_similar_keywords(df[df['Beer_name'] == beer_name])

    return "\n".join([f"{k}: {v}" for k, v in keywords.items()])

beer_names = pre_apply_beer_Wired_iStout['Beer_name'].unique()
sentiments = ["Positive", "Negative"]

interface = gr.Interface(
    show_keywords,
    [
      gr.Dropdown(beer_names, label="Beer_name", info="Choose the beer you want to see the summary keyword."),
      gr.Dropdown(choices=sentiments, label="Sentiment", info="Choose Positive or Negative sentiment."),
    ],
    outputs="text",
)

interface.launch()
